In [ ]:
import torch
import torchvision.datasets
import torchvision.transforms as transforms
from torch import nn
from torch.utils.data import DataLoader
import torch.nn.functional as F

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd "/content/drive/MyDrive/Fall-2022/ESE-546/Project_updated"

/content/drive/MyDrive/Fall-2022/ESE-546/Project_updated


# Loading the CIFAR-10 dataset

We will be running our entire analysis on CIFAR-10 dataset. We will train the all-cnn model on it and once trained, this model will be targetted for adversarial attacks.

In [ ]:
use_cuda = True
image_nc = 3
batch_size = 256

# Define what device we are using
print("CUDA Available: ", torch.cuda.is_available())
device = torch.device("cuda" if (use_cuda and torch.cuda.is_available()) else "cpu")

train_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

test_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Load dataset
cifar10_trainset = torchvision.datasets.CIFAR10('./dataset', train=True, transform=train_transforms, download=True)
cifar10_testset = torchvision.datasets.CIFAR10('./dataset', train=False, transform=test_transforms, download=True)

# Create a dataloader for train and test datasets
train_dataloader = DataLoader(cifar10_trainset, batch_size=batch_size, shuffle=True, num_workers=1)
test_dataloader = DataLoader(cifar10_testset, batch_size=batch_size, shuffle=True, num_workers=1)

CUDA Available:  True
Files already downloaded and verified
Files already downloaded and verified


In [ ]:
class BaseModel_All_CNN(nn.Module):
    def __init__(self, c1=96, c2= 192):
        super().__init__()

        def convbn(input, output, kernel_size, stride=1, padding=0):
            return nn.Sequential(
                nn.Conv2d(input, output, kernel_size, stride=stride, padding=padding),
                nn.ReLU(True),
                nn.BatchNorm2d(output))

        self.net = nn.Sequential(
            nn.Dropout(0.2),
            convbn(3, c1, 3, 1, 1),
            convbn(c1, c1, 3, 1, 1),
            convbn(c1, c1, 3, 2, 1),

            nn.Dropout(0.5),
            convbn(c1, c2, 3, 1, 1),
            convbn(c2, c2, 3, 1, 1),
            convbn(c2, c2, 3, 2, 1),

            nn.Dropout(0.5),
            convbn(c2, c2, 3, 1, 1),
            convbn(c2, c2, 3, 1, 1),
            convbn(c2, 10, 1, 1),

            nn.AvgPool2d(8))

        print('Num parameters: ', sum([params.numel() for params in self.net.parameters()]))

    def forward(self, x):
        x = self.net(x)
        x = x.view(-1, 10)
        return x

In [ ]:
# training the target model
target_model = BaseModel_All_CNN().to(device)
target_model.train()
opt_model = torch.optim.Adam(target_model.parameters(), lr=0.01, weight_decay=1e-5)
epochs = 40
for epoch in range(epochs):
    loss_epoch = 0
    if epoch == 20:
        for g in opt_model.param_groups:
          g['lr'] = 0.001
    for i, data in enumerate(train_dataloader, 0):
        train_imgs, train_labels = data
        train_imgs, train_labels = train_imgs.to(device), train_labels.to(device)
        logits_model = target_model(train_imgs)
        loss_model = F.cross_entropy(logits_model, train_labels)
        loss_epoch += loss_model
        opt_model.zero_grad()
        loss_model.backward()
        opt_model.step()

    print('loss in epoch %d: %f' % (epoch, loss_epoch.item()))

# save model
targeted_model_file_name = './CIFAR10_target_model.pth'
torch.save(target_model.state_dict(), targeted_model_file_name)
target_model.eval()

Num parameters:  1667166
loss in epoch 0: 347.238251
loss in epoch 1: 262.244293
loss in epoch 2: 210.256516
loss in epoch 3: 181.061890
loss in epoch 4: 156.946838
loss in epoch 5: 139.695755
loss in epoch 6: 128.935760
loss in epoch 7: 121.425812
loss in epoch 8: 114.716202
loss in epoch 9: 109.435890
loss in epoch 10: 105.339836
loss in epoch 11: 101.212662
loss in epoch 12: 97.968475
loss in epoch 13: 94.792564
loss in epoch 14: 92.408791
loss in epoch 15: 91.843773
loss in epoch 16: 89.339005
loss in epoch 17: 86.249924
loss in epoch 18: 86.221329
loss in epoch 19: 83.926117
loss in epoch 20: 65.911072
loss in epoch 21: 57.720028
loss in epoch 22: 54.040348
loss in epoch 23: 51.898479
loss in epoch 24: 49.357708
loss in epoch 25: 47.984421
loss in epoch 26: 46.330807
loss in epoch 27: 44.603626
loss in epoch 28: 42.856987
loss in epoch 29: 42.750599
loss in epoch 30: 40.717289
loss in epoch 31: 39.383335
loss in epoch 32: 38.832455
loss in epoch 33: 37.201641
loss in epoch 34: 36.

BaseModel_All_CNN(
  (net): Sequential(
    (0): Dropout(p=0.2, inplace=False)
    (1): Sequential(
      (0): Conv2d(3, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (2): Sequential(
      (0): Conv2d(96, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (3): Sequential(
      (0): Conv2d(96, 96, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (4): Dropout(p=0.5, inplace=False)
    (5): Sequential(
      (0): Conv2d(96, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running

# Load the saved model and check validation accuracy

We will load our weights to verify and check the validation accuracy on this model.

In [ ]:
TARGET_MODEL_PATH = './CIFAR10_target_model.pth'

# Load trained base model
target_model = BaseModel_All_CNN().to(device)
target_model.load_state_dict(torch.load(TARGET_MODEL_PATH))

Num parameters:  1667166


<All keys matched successfully>

In [ ]:
num_correct = 0
for i, data in enumerate(test_dataloader, 0):
    test_img, test_label = data
    test_img, test_label = test_img.to(device), test_label.to(device)

    pred_lab = torch.argmax(target_model(test_img), 1)
    num_correct += torch.sum(pred_lab==test_label,0)
    
print('Base model accuracy in testing set: %f\n'%(num_correct.item()/len(cifar10_testset)))

Base model accuracy in testing set: 0.876700

